In [9]:
%run func_def.ipynb

imports loaded.
get_height(item)
get_weight(item)
get_reach(item)
get_std_reach(item, height, reach_list, height_list)
get_age(item)
get_std_age(item, age_list)
get_win_perc(name, date, df)
get_lose_streak_data_frame(name, date, df)
get_win_streak_data_frame(name, date, df)
get_streak_data_frame(name, date, df)
get_win_streak(name, date, df)
get_lose_streak(name, date, df)
get_fighter_stats(name, data )
add_columns(data, column_list)
get_stat(index, data, stat='height', fo='')
get_swaped_row(index, data)
get_wins(combo, data)
get_wins_ratio(combo, data)
get_r_score(X, y)
get_sig_features(data, sig_r)
set_train_test(X_columns, y_columns, data, test_size=0.3, valid_size=0.3)
set_knn_reg(X_columns, y_columns, data, n=10)
set_predictions(model, data, X_columns, y_columns)
get_rmse(act, pred)
set_desicion_tree_clf(X_columns, y_columns, data, tree_depth)
set_forest_clf(X_columns, y_columns, n_estimators, tree_depth, data)
get_accuracy(act, pred)
get_fighters_stats(fighter, opponent, data)
se

# <span style='color:green'> [done] </span> Loading data frames:

In [25]:
loc = '../data/'
file = 'ufc_data_trained.csv'
data = pd.read_csv(loc+file)
data = data[data.columns[1:]]
data.tail(3)

,set,stance_win_ratio,win_perc_dif,win_strk_dif,lose_strk_dif,kd_dif,str_dif,td_dif,sub_dif,result,result_pred
6487,train,-0.160261,-2.063384,-0.575708,2.156259,-0.000995,0.126491,-1.597572,-0.013366,lose,lose
6488,train,2.207079,-0.016271,-0.004540,-1.072901,-0.000995,-0.174504,1.558550,-0.013366,win,win
6489,train,1.461511,0.111674,-0.004540,-2.149288,1.351338,0.126491,-0.019511,-0.013366,win,win


In [ ]:
fighter = 'Kelvin Gastelum'
opponent = 'Nassourdine Imavov'
df = pd.concat([df,set_new_fight(fighter=fighter, opponent=opponent, data=df)], ignore_index=True)
df.tail(1)

## <span style='color:green'> [done] </span> Load fighters data_frame:

In [10]:
loc = '../data/'
file = 'ufc_fighters_stats_processed.csv'
stats = pd.read_csv(loc+file)
stats = stats[stats.columns[1:]]
print('Data Frame loaded: '+str(stats.shape))
stats.tail(3)

Data Frame loaded: (11198, 17)


,name,wins,losses,draws,height,weight,reach,stance,age,slpm,str. acc.,sapm,str. def,td avg.,td acc.,td def.,sub. avg.
11195,Carlos Zevallos,3,0.0,-0,182.88,92.25,187.37,Orthodox,37.31,4.36,0.65,2.28,0.68,0.0,0.0,1.00,0.0
11196,Zach Zane,10,7.0,0,170.18,65.25,175.26,Southpaw,33.00,0.87,0.56,6.67,0.20,0.0,0.0,0.26,0.0
11197,Roger Zapata,4,0.0,-0,180.34,76.50,184.77,Southpaw,36.00,2.60,0.51,2.13,0.36,0.0,0.0,0.81,1.4


## <span style='color:green'> [done] </span> Prepare main data_frame:

In [11]:
loc = "../data/"
file = 'ufc_data_features.csv'
df = pd.read_csv(loc+file)
df = df[df.columns[1:]]
df.date = pd.to_datetime(df.date)
print('Data Frame loaded: '+str(df.shape))
df.tail(3)

Data Frame loaded: (6793, 22)


,result,fighter,opponent,date,weight class,method,kd_dif,str_dif,td_dif,sub_dif,td avg._dif,td acc._dif,td def._dif,sub. avg._dif,slpm_dif,str. acc._dif,sapm_dif,str. def_dif,stance_win_ratio,win_perc_dif,win_strk_dif,lose_strk_dif
6790,lose,Kailin Curran,Aleksandra Albu,2017-07-29,Women's Strawweight,U-DEC:,-0.000995,0.126491,-1.597572,-0.013366,-0.159903,0.226867,-0.572982,0.195941,-0.626083,-0.350298,-0.477706,1.235656,-0.160261,-2.063384,-0.575708,2.156259
6791,win,Jarred Brooks,Eric Shelton,2017-07-29,Flyweight,S-DEC:,-0.000995,-0.174504,1.558550,-0.013366,0.941938,0.191864,0.734104,0.386715,-0.317445,0.299038,-0.118606,-0.111192,2.207079,-0.016271,-0.004540,-1.072901
6792,win,Drew Dober,Joshua Burkman,2017-07-29,Lightweight,KO/TKO:Punch,1.351338,0.126491,-0.019511,-0.013366,-1.000196,-0.683207,-0.640012,-0.185607,1.087173,-0.269131,0.642198,-0.027014,1.461511,0.111674,-0.004540,-2.149288


## <span style='color:green'> [done] </span> Clean Outliers:

In [12]:
original_size = len(df)
X = df.columns[10:len(df.columns)]

for feature in X:
    df = df[df[feature] <= 3]
    
print('Rows deleted: %d' %(original_size-len(df)))

Rows deleted: 387


In [13]:
df = (df[['date','result','fighter','opponent','win_perc_dif','win_strk_dif','lose_strk_dif','td avg._dif','sub. avg._dif','sapm_dif','slpm_dif','kd_dif','str_dif','td_dif','sub_dif']]).copy()
df = df.dropna().reset_index(drop=True)
print('Columns sorted:')
df.tail(3)

Columns sorted:


,date,result,fighter,opponent,win_perc_dif,win_strk_dif,lose_strk_dif,td avg._dif,sub. avg._dif,sapm_dif,slpm_dif,kd_dif,str_dif,td_dif,sub_dif
6403,2017-07-29,lose,Kailin Curran,Aleksandra Albu,-2.063384,-0.575708,2.156259,-0.159903,0.195941,-0.477706,-0.626083,-0.000995,0.126491,-1.597572,-0.013366
6404,2017-07-29,win,Jarred Brooks,Eric Shelton,-0.016271,-0.004540,-1.072901,0.941938,0.386715,-0.118606,-0.317445,-0.000995,-0.174504,1.558550,-0.013366
6405,2017-07-29,win,Drew Dober,Joshua Burkman,0.111674,-0.004540,-2.149288,-1.000196,-0.185607,0.642198,1.087173,1.351338,0.126491,-0.019511,-0.013366


In [14]:
X_columns = X
y = ['kd_dif','str_dif','td_dif','sub_dif']

# <span style='color:green'> **[done]** </span> Main():

In [15]:
X_columns = ['sapm_dif','slpm_dif','win_perc_dif','win_strk_dif','lose_strk_dif','td avg._dif','sub. avg._dif']
y = ['kd_dif','str_dif','td_dif','sub_dif']

## <span style='color:green'> **[done]** </span> Pick a Fight: inserd Fighter & Opponent

In [16]:
fighter = 'Kelvin Gastelum'
opponent = 'Nassourdine Imavov'
df = pd.concat([df,set_new_fight(fighter=fighter, opponent=opponent, data=df)], ignore_index=True)
df.tail(1)

,date,result,fighter,opponent,win_perc_dif,win_strk_dif,lose_strk_dif,td avg._dif,sub. avg._dif,sapm_dif,slpm_dif,kd_dif,str_dif,td_dif,sub_dif
6406,2023-01-22,None,Kelvin Gastelum,Nassourdine Imavov,-0.221053,-3.0,2.0,0.1,-2.3,0.81,-0.55,None,None,None,None


### <span style='color:green'> **[done]** </span> Predict fight stats:

In [17]:
# test making prediction of stats by creating only both fighters data:
micro_df = (df[(df['fighter'] == fighter)|(df['opponent'] == fighter)|(df['fighter'] == opponent)|(df['opponent'] == opponent)].reset_index(drop=True)).copy()
#micro_df = set_train_test(X_columns=X_columns, y_columns=y, data=micro_df, test_size=0.3, valid_size=0.3)
micro_df.tail(3)

,date,result,fighter,opponent,win_perc_dif,win_strk_dif,lose_strk_dif,td avg._dif,sub. avg._dif,sapm_dif,slpm_dif,kd_dif,str_dif,td_dif,sub_dif
22,2017-11-25,lose,Michael Bisping,Kelvin Gastelum,-0.060215,0.001615,0.006402,-0.007068,0.197523,-0.325881,0.509521,-1.343474,-0.265609,-0.018065,0.025311
23,2019-11-02,lose,Kelvin Gastelum,Darren Till,-0.009111,0.001615,-1.070893,0.299562,0.102182,0.131993,0.806828,0.011068,0.14842,-0.018065,0.025311
24,2023-01-10,None,Kelvin Gastelum,Nassourdine Imavov,-0.221053,-3.000000,2.000000,0.100000,-2.300000,0.810000,-0.550000,None,None,None,None


In [17]:
df = set_train_test(X_columns=X_columns, y_columns=y, data=df, test_size=0.3, valid_size=0.3)
knn_reg = set_knn_reg(X_columns=X_columns, y_columns=y, data=df, n=200)
df = set_predictions(model=knn_reg, data=df, X_columns=X_columns, y_columns=y)
knn_reg

Data split: Data=(6407, 16), train_set=(3134, 7), validation_set=(1334, 7), test_set=(1938, 7)


KNeighborsRegressor(algorithm='ball_tree', n_neighbors=200)

In [19]:
df.tail(1)

,set,date,result,fighter,opponent,win_perc_dif,win_strk_dif,lose_strk_dif,td avg._dif,sub. avg._dif,sapm_dif,slpm_dif,kd_dif,str_dif,td_dif,sub_dif,kd_dif_pred,str_dif_pred,td_dif_pred,sub_dif_pred
6342,new,2023-01-10,None,Kelvin Gastelum,Nassourdine Imavov,-0.221053,-3.0,2.0,0.1,-2.3,0.81,-0.55,None,None,None,None,0.038159,0.038159,0.038159,0.038159


### <span style='color:green'> **[done]** </span> Predict Winner:

In [21]:
X_columns = ['win_perc_dif', 'win_strk_dif', 'lose_strk_dif', 'td avg._dif',
             'sub. avg._dif', 'sapm_dif', 'slpm_dif','sub_dif_pred',
             'kd_dif_pred', 'str_dif_pred', 'td_dif_pred']

#num_of_features = len(X_columns)
num_of_features = 3
y = 'result'
forest_clf = set_forest_clf(X_columns=X_columns, y_columns=y, data=df,tree_depth=num_of_features, n_estimators=50 )
df = set_predictions(model=forest_clf, data=df, X_columns=X_columns, y_columns=[y])
forest_clf

RandomForestClassifier(max_depth=3, n_estimators=50, random_state=0)

#### <span style='color:green'> **[done]** </span> Final Prediction:

In [22]:
print('--------------------------------------')
print('%s vs %s:' %(fighter, opponent))
print('--------------------------------------')
print(get_fighters_stats(fighter=fighter,opponent=opponent, data=stats))
print('--------------------------------------')
print('Winner: ' + get_winner())
print('Pridiction Accuracy: ' + str(get_accuracy(act=df['result'][df.set=='test'], pred=df['result_pred'][df.set=='test'])))
print('--------------------------------------')

--------------------------------------
Kelvin Gastelum vs Nassourdine Imavov:
--------------------------------------
          Kelvin Gastelum Nassourdine Imavov
wins                   17                 12
losses                8.0                3.0
draws                   0                  0
height             175.26              190.5
weight              83.25              83.25
reach              180.34              190.5
stance           Southpaw           Orthodox
age                  31.0               27.0
slpm                 3.53               4.08
str. acc.            0.41               0.54
sapm                 3.24               2.43
str. def             0.57               0.62
td avg.              1.05               0.95
td acc.              0.32               0.23
td def.              0.62               0.76
sub. avg.             0.1                2.4
--------------------------------------
Winner: Nassourdine Imavov
Pridiction Accuracy: 0.6140350877192983
------------